In [2]:
pip install tensorflow_hub

In [5]:
pip install deeppavlov

In [4]:
pip install pandas

In [6]:
pip install numpy

In [ ]:
pip install tensorflow==1.15.0

In [8]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

In [10]:
import pandas as pd
import numpy as np

In [11]:
from scipy.spatial.distance import cosine

In [ ]:
elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
data = pd.read_csv('data_for_w2v.csv', encoding='utf-16-be')
data

In [17]:
data = pd.read_csv('all_courses_clean.csv')

In [18]:
# data = data.drop({'Unnamed: 0'}, axis=1)
data

,Unnamed: 0,content_id,all_info
0,0,000249F5-15E7-40FA-A835-CB6D34EF355E,контрольный вопрос и задание инструмент интер...
1,1,0008020A-E57E-40BD-9196-AA20637936E1,unitx purchasing and storage от автор unit ii ...
2,2,000DDF9E-81D8-4612-A4DF-07E70705E0C9,основной вид и характеристика электроинструме...
3,3,000E9424-369E-43DD-BC66-D46A6E7A995C,содержание представление и умение школьный т...
4,4,00114246-96FC-4155-A563-EA55C52A5596,тема расследование нарушение правило техника ...
...,...,...,...
30260,30260,FFF3CE71-073B-45D1-8BFA-09A27CA28140,корпоративный социальный ответственность инос...
30261,30261,FFF4D06F-38D7-4AE6-8668-5C3F6EC5C978,уральский федеральный округ глава методологи...
30262,30262,FFF8195F-8F00-407D-A656-5B6D4D90BC90,глава проблема правовой регулирование социаль...
30263,30263,FFFA7927-2850-4A29-94D1-7AFEB54D0084,контрольный вопрос и задание глава латеральны...


In [19]:
data = data.drop_duplicates(subset='all_info')
data

,Unnamed: 0,content_id,all_info
0,0,000249F5-15E7-40FA-A835-CB6D34EF355E,контрольный вопрос и задание инструмент интер...
1,1,0008020A-E57E-40BD-9196-AA20637936E1,unitx purchasing and storage от автор unit ii ...
2,2,000DDF9E-81D8-4612-A4DF-07E70705E0C9,основной вид и характеристика электроинструме...
3,3,000E9424-369E-43DD-BC66-D46A6E7A995C,содержание представление и умение школьный т...
4,4,00114246-96FC-4155-A563-EA55C52A5596,тема расследование нарушение правило техника ...
...,...,...,...
30233,30233,FFB56DEF-DB07-479C-8B71-66E41E662F20,требование к документ представлять для соверш...
30245,30245,FFD2F38B-96FF-4AE8-8D50-6E443AED451B,заключение приложение новый издание подисцип...
30252,30252,FFE90073-7D7D-405A-A6E0-5F504A15CA6B,сельскохозяйственный район раздел iv особенно...
30255,30255,FFEC7B85-3062-44DE-B240-F1E81670789A,правописание согласный в корн слово слитный ...


In [12]:
vectorize = pd.DataFrame(columns=['vector'])

In [31]:
data['Unnamed: 0'].values

array([    0,     1,     2, ..., 30252, 30255, 30262])

In [ ]:
def to_vector(words):
  request_vect = np.zeros((1, 1024))
  vect_word = elmo([[words]])
  for i in vect_word:
    request_vect += i
  return request_vect

In [ ]:
dt = []
for i in data['Unnamed: 0']:
    if not i % 100: print(i)
    dt.append(elmo([[data.all_info[i]]]))
print('ok')

In [ ]:
x = elmo([[data.all_info[0]]])

In [ ]:
np.sum(x)

6.7003794

In [23]:
vectorize = pd.DataFrame(columns=['vector'])

In [24]:
for i in dt:
  vectorize = vectorize.append({'vector': i}, ignore_index=True)
vectorize

,vector
0,"[[0.26895428, 0.30151373, 0.5491725, 0.344743,..."
1,"[[-0.4643876, -0.078544065, -0.23469087, 0.183..."
2,"[[-0.5667841, 0.4193952, 0.25175086, 0.3133003..."
3,"[[-0.43150905, -0.5264995, -0.12793583, 0.2546..."
4,"[[-0.44764304, -0.07800484, 0.20476314, 0.7870..."
...,...
11012,"[[-0.16875425, -0.23142013, 0.08481322, 0.1283..."
11013,"[[-0.48557043, -0.37868384, 0.28364906, 0.8441..."
11014,"[[-0.25701356, -0.61224544, -0.09269293, 0.133..."
11015,"[[-0.45966884, -0.29324463, 0.5415814, 0.41439..."


In [32]:
vectorize['Unnamed: 0'] = data['Unnamed: 0'].values
vectorize

,vector,Unnamed: 0
0,"[[0.26895428, 0.30151373, 0.5491725, 0.344743,...",0
1,"[[-0.4643876, -0.078544065, -0.23469087, 0.183...",1
2,"[[-0.5667841, 0.4193952, 0.25175086, 0.3133003...",2
3,"[[-0.43150905, -0.5264995, -0.12793583, 0.2546...",3
4,"[[-0.44764304, -0.07800484, 0.20476314, 0.7870...",4
...,...,...
11012,"[[-0.16875425, -0.23142013, 0.08481322, 0.1283...",30233
11013,"[[-0.48557043, -0.37868384, 0.28364906, 0.8441...",30245
11014,"[[-0.25701356, -0.61224544, -0.09269293, 0.133...",30252
11015,"[[-0.45966884, -0.29324463, 0.5415814, 0.41439...",30255


In [34]:
new_vectorize = data.set_index('Unnamed: 0').join(vectorize.set_index('Unnamed: 0')).drop({'all_info'}, axis=1).reset_index()
new_vectorize

,Unnamed: 0,content_id,vector
0,0,000249F5-15E7-40FA-A835-CB6D34EF355E,"[[0.26895428, 0.30151373, 0.5491725, 0.344743,..."
1,1,0008020A-E57E-40BD-9196-AA20637936E1,"[[-0.4643876, -0.078544065, -0.23469087, 0.183..."
2,2,000DDF9E-81D8-4612-A4DF-07E70705E0C9,"[[-0.5667841, 0.4193952, 0.25175086, 0.3133003..."
3,3,000E9424-369E-43DD-BC66-D46A6E7A995C,"[[-0.43150905, -0.5264995, -0.12793583, 0.2546..."
4,4,00114246-96FC-4155-A563-EA55C52A5596,"[[-0.44764304, -0.07800484, 0.20476314, 0.7870..."
...,...,...,...
11012,30233,FFB56DEF-DB07-479C-8B71-66E41E662F20,"[[-0.16875425, -0.23142013, 0.08481322, 0.1283..."
11013,30245,FFD2F38B-96FF-4AE8-8D50-6E443AED451B,"[[-0.48557043, -0.37868384, 0.28364906, 0.8441..."
11014,30252,FFE90073-7D7D-405A-A6E0-5F504A15CA6B,"[[-0.25701356, -0.61224544, -0.09269293, 0.133..."
11015,30255,FFEC7B85-3062-44DE-B240-F1E81670789A,"[[-0.45966884, -0.29324463, 0.5415814, 0.41439..."


In [35]:
new_vectorize.to_csv('elmo_vectorize.csv')

In [71]:
sort_dict = lambda dictn: {key: value for key, value in sorted(dictn.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
courses  = pd.read_csv('contents.csv', encoding='utf-16')
courses

In [ ]:
contents = pd.read_csv('courses.csv', encoding='utf-16')
contents

In [39]:
new_data = new_vectorize.set_index('content_id').join(courses[['content_id', 'normal_name', 'bdescription']].set_index('content_id')).reset_index().drop({'Unnamed: 0'}, axis=1)
new_data

,content_id,vector,normal_name,bdescription
0,000249F5-15E7-40FA-A835-CB6D34EF355E,"[[0.26895428, 0.30151373, 0.5491725, 0.344743,...",Маркетинг в отраслях и сферах деятельности,Теории маркетинга посвящено множество достойны...
1,0008020A-E57E-40BD-9196-AA20637936E1,"[[-0.4643876, -0.078544065, -0.23469087, 0.183...",Деловой английский язык для ресторанного бизне...,Настоящее учебное пособие предназначено для ст...
2,000DDF9E-81D8-4612-A4DF-07E70705E0C9,"[[-0.5667841, 0.4193952, 0.25175086, 0.3133003...",Электрический привод,В учебнике даны основные сведения об электропр...
3,000E9424-369E-43DD-BC66-D46A6E7A995C,"[[-0.43150905, -0.5264995, -0.12793583, 0.2546...",Дошкольная педагогика. Развитие пространственн...,В издании представлена методика диагностики пр...
4,00114246-96FC-4155-A563-EA55C52A5596,"[[-0.44764304, -0.07800484, 0.20476314, 0.7870...",Криминалистика: тактика и методика. Задачник,Главная цель настоящего учебного пособия — пом...
...,...,...,...,...
11012,FFB56DEF-DB07-479C-8B71-66E41E662F20,"[[-0.16875425, -0.23142013, 0.08481322, 0.1283...",Нотариат,Учебник подготовлен с учетом новейших изменени...
11013,FFD2F38B-96FF-4AE8-8D50-6E443AED451B,"[[-0.48557043, -0.37868384, 0.28364906, 0.8441...",Обучение чтению на иностранном языке в совреме...,Во втором издании монографии «Обучение чтению ...
11014,FFE90073-7D7D-405A-A6E0-5F504A15CA6B,"[[-0.25701356, -0.61224544, -0.09269293, 0.133...",Территориальное планирование,"В учебнике освещаются теоретические основы, со..."
11015,FFEC7B85-3062-44DE-B240-F1E81670789A,"[[-0.45966884, -0.29324463, 0.5415814, 0.41439...",Русский язык и культура речи,Учебник построен в соответствии с Федеральным ...


In [77]:
with open('elmo_vec_np.npy', 'wb') as file:
  np.save(file, new_data)

In [82]:
with open('test.npy', 'wb') as file:
  np.save(file, new_data['vector'])

In [83]:
a = np.load('test.npy')

ValueError: ignored

In [69]:
new_data.to_csv('elmo_vector.csv')

In [ ]:
for i in new_data["normal_name"].values:
  if 'математика' in i:
    print(i)


In [65]:
new_data[new_data.normal_name == 'Прикладная математика']

,content_id,vector,normal_name,bdescription
522,047D5DBE-DCE5-4AA4-8366-D43B4655E9B5,"[[-0.14624117, 0.0017065108, -0.07420351, 0.06...",Прикладная математика,В курсе рассмотрены основные методы нелинейног...
10834,F251B781-A1F0-4649-953D-DC0E0E86A041,"[[-0.030289322, -0.9246818, 0.34354544, 0.3424...",Прикладная математика,В курсе рассмотрены основные методы нелинейног...


In [48]:
new_data['normal_name'][11015], new_data['content_id'][11015]

('Русский язык и культура речи', 'FFEC7B85-3062-44DE-B240-F1E81670789A')

In [72]:
def recommendation_system_with_cosine(request):
    simularity_course = {}
    req_vector = new_vectorize[new_vectorize.content_id == request].vector.values[0]
    for i, v in new_vectorize.iterrows():
        simularity_course[v.content_id] = cosine(req_vector, v.vector)
    simularity_course = sort_dict(simularity_course)

    return simularity_course

In [73]:
req = '047D5DBE-DCE5-4AA4-8366-D43B4655E9B5'

In [74]:
rec = recommendation_system_with_cosine(req)
# rec

In [68]:
for i in list(rec)[:6]:
  print(courses[courses.content_id == i].normal_name.values[0])

Прикладная математика
Логистика и управление цепями поставок
Логистика и управление цепями поставок
Комплексный экономический анализ хозяйственной деятельности в 2 ч. Часть 1
Маркетинг туристских услуг
Маркетинг туристских услуг


In [1]:
# print(courses[courses.content_id == req].normal_name, '\n')
s = set()
for i in rec:
  a = courses[courses.content_id == i].normal_name.values[0]
  if a not in s:
    print(a)
    s.add(a)
  if len(s) == 6:
    break

NameError: ignored

In [ ]:
elmo.